In [1]:
# pip install python-bidi
# pip install hazm

In [2]:
from tqdm.notebook import tqdm
import numpy as np
import re
import string
from hazm import word_tokenize
from hazm import sent_tokenize
from hazm import Normalizer
from collections import Counter

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Learn about the text before starting to clean and preprocess it.

In [3]:
with open('./persian_combined_texts.txt', 'r', encoding="utf-8") as f:
    dirty_corp = f.read()
dirty_corp = dirty_corp.split('^^^^^^')
dirty_corp = dirty_corp[:-1]
print(f'Num of texts: {len(dirty_corp)}')

Num of texts: 1381


Please note that the actuall number of texts here is 2000. It is just because when converting pdf files to  word files we sometimes combined more than one pdf file in one word file

In [4]:
print(dirty_corp[1][1100:2100])

شکیل شده ایم که خود با سرعتی سرسام آور در حرکت دورانی کاینات شریکند. آنهم در حالی است که اگر از فضای خالی موجود درکلٌ مواد کره زمین صرف نظر کنیم» اندازهٌ مجموع اين ارواح زندهٌ سیار از یک توپ فوتبال بزرگتر نخواهد بود.  گرچه خانه ازپای بست سست و ویران است» لاکن بنیانگذار آداب و سنن کهن آسیا یعنی زرتشت/بودا/ابراهیم خلیل بزرگمرد بی نظیر و بی مثال تاریخ عهد باستان بوده است.  دانش زرتشتیها از زرتشت تنها به اندازةٌ سر کوه یخی است که از اقیانوس تاریخ سر بر آورده است؛ پس باید دید دیگران از وی چه می گویند.   صفحه: 4  سیمای واقعی زرتشت تاریخی  (اتا آعطیناک الکوش)  دخمه گانوماته بردیه (زرتشت) در روستای سکاوند بخش هرسین کرمانشاهان  آب زنید راه راء هین که نسگار می رسد سس مژده دهید باغ را بوی بهار می رسد   صفحه: 5  معنی لفظی نام عجم و پیوند ناگسستنی تاریخی آن با جمشید و جم  می دانیم كلمةٌ عجم در زبان عرب به معنی کسی که دارای زبان غیر فصیح و لکنت داراست می باشد و از همینجاست که محمد بن جریر طبری به پیروی از افواه عامه بابک خرمدین را تحت این لقب یعنی دارای زبان لکنت دار آورده است» در صورتیکه بابک خرمدی

In [5]:
#Count the number of words in the file
num_words = sum(len(doc.split()) for doc in dirty_corp)
#num_words
print("The file contains", num_words, "words.")

The file contains 110037959 words.


In [6]:
word_counts = Counter() 
for line in dirty_corp:
    words = word_tokenize(line.strip())   # tokenize each line into words
    word_counts.update(words)

most_common_words = word_counts.most_common(10)
print("The 10 most common words in the file are:")
for word, count in most_common_words:
    print(word, count)

The 10 most common words in the file are:
و 4908774
. 4586423
در 2387302
که 2309124
از 2260401
» 2034824
به 1948159
: 1833595
را 1601446
( 1213648


Because the text is still unclean, we get the above most common words, which is a mess))

So, we will clean the text and run this code later.

In [7]:
import string
import re

class TextCleaner():
    """
    Persian text cleaner
    """
    def __init__(self):
        self.min_ascii = '0600'
        self.max_ascii = '06FF'
        self.punctuation = string.punctuation + '،' + '؛' + '؟' + '؛' + '۔' + '»' + '«' + '-'
        self.one_space_regex = r"\s((\s)(\s+)?)?"
        self.text = None
        self.dict_punct = dict(zip(list(self.punctuation), np.repeat(' ', len(self.punctuation))))
    
    def remove_punct(self, text):
        # Remove patterns of the form "number: صفحه"
        text = re.sub(r'صفحه\s*:\s*\d+', '', text)
        # Remove all other punctuation
        table = str.maketrans(self.dict_punct)
        text = text.translate(table)
        return text
    
    def remove_num(self, text):
        # Remove all numbers
        num_pattern = r'[\u06F0-\u06F9]'
        text = re.sub(num_pattern, ' ', text)
        return text
    
    def remove_spaces(self, text):
        # Remove extra spaces
        try:
            text = re.sub(self.one_space_regex, ' ', text)    
            text = text if text[0] != ' ' else text[1:]
            text = text if text[-1] != ' ' else text[:-1]
            return text
        
        except IndexError as e:
            return ''
    
    def is_fa_token(self, token):
        # Check if token is in Persian language
        for ch in set(token):
            if ord(ch) < int(self.min_ascii, 16) or ord(ch) > int(self.max_ascii, 16):
                return False
        return True

    def remove_foreign_lang(self, text):
        # Remove non-Persian tokens
        clean_text = ''
        for token in text.split():
            if self.is_fa_token(token):
                clean_text += ' ' + token
        return clean_text[1:]
    
    
    def clean_text(self, text):
        text = self.remove_num(text)
        text = self.remove_punct(text)
        text = self.remove_spaces(text)
        text = self.remove_foreign_lang(text)
        text = self.remove_spaces(text)
        return text

In [8]:
cleaned_texts = []

text_cleaner = TextCleaner()

for text in dirty_corp:
    cleaned_text = text_cleaner.clean_text(text)
    cleaned_texts.append(cleaned_text)

In [9]:
len(cleaned_texts)

1381

In [10]:
#print(cleaned_texts[1][1:100])

## Removing Stopwords

[Stopwords were taken from here]("https://github.com/mhbashari/awesome-persian-nlp-ir")

In [11]:
# Load the stopwords from the text file
with open("Persian-stopwords.txt", "r", encoding="utf-8") as f:
    stop_words = [line.strip() for line in f]

In [12]:
#stop_words

In [13]:
cleaned_corp= []

for text in cleaned_texts:
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    cleaned_corp.append(' '.join(filtered_words))

In [14]:
word_counts = Counter() 
for line in cleaned_corp:
    words = word_tokenize(line.strip())   # tokenize each line into words
    word_counts.update(words)

most_common_words = word_counts.most_common(10)
print("The 10 most common words in the file are:")
for word, count in most_common_words:
    print(word, count)

The 10 most common words in the file are:
سال 173800
ایران 168728
کار 142945
دست 137521
بن 132340
تاریخ 110698
کتاب 107934
قرار 94694
شاه 84279
نظر 82769


## Normalization

In [15]:
from hazm import *
normalizer = Normalizer()

normalized_texts = []
for text in cleaned_corp: #change it to cleaned_corp after saving the coprus with stopwords
    normalized_text= normalizer.normalize(text)
    normalized_texts.append(normalized_text)

## Stemming & Lemmatization 

In [16]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

stemmed_lemmatized_texts = []

for text in normalized_texts:
    stemmed_text = [stemmer.stem(word) for word in text.split()]
    lemmatized_text = [lemmatizer.lemmatize(word) for word in stemmed_text]
    stemmed_lemmatized_text = ' '.join(lemmatized_text)
    stemmed_lemmatized_texts.append(stemmed_lemmatized_text)

### Replace pronouns, and determiners with tokens

In [17]:
import re
import nltk


def replace_pronouns(text):
    # Replace common Persian pronouns with a specific token
    pronoun_mapping = {
        'او': '<PRON>',
        'اون': '<PRON>',
        'اوه': '<PRON>',
        'شما': '<PRON>',
        'اوها': '<PRON>',

        # Add more pronouns as needed
    }
    tokens = nltk.word_tokenize(text)
    replaced_tokens = [pronoun_mapping.get(token, token) for token in tokens]
    return ' '.join(replaced_tokens)

def replace_determiners(text):
    # Replace common Persian determiners with a specific token
    determiner_mapping = {
        'یک': '<DET>',
        'یه': '<DET>',
        'این': '<DET>',
        'اون': '<DET>',
        'همه': '<DET>',
        'هر': '<DET>',
        'همین': '<DET>',
        
        # Add more determiners as needed
    }
    tokens = nltk.word_tokenize(text)
    replaced_tokens = [determiner_mapping.get(token, token) for token in tokens]
    return ' '.join(replaced_tokens)

In [18]:
cleaned_texts = []
for text in stemmed_lemmatized_texts:
    cleaned_text = replace_pronouns(text)
    cleaned_text = replace_determiners(cleaned_text)
    cleaned_texts.append(cleaned_text)

# The cleaned_texts list now contains the cleaned and replaced texts

### An overview how the text looks before and after cleaning

In [19]:
initial_text = dirty_corp[2][1000:2000]
final_text = cleaned_texts[2][1000:2000]
print(f'Before: {initial_text}')
print('\n')
print(f'After: {final_text}')

Before: ای غلط نظامیان ذکر می‌کنند اما امروز عامل دیگری نیز به این دو عامل اضافه شده است. افکاز عمومی, گاهی افکار عمومی جنگ را می‌طلید.  بررسی تاریخ جنگ‌ها نشان می‌دهد پس از مدتی تخاصم لفظی و یا دست اندازی‌های محدود. شعله‌ی جنگ ناگهان بین دو يا چند ملت شعله‌ور و کشورء قاره و یا دنیایی به آتش کشیده می‌شود.  دلیل بروز نبردها ساده و گاهی کم اهمیت جلوه می‌کند اما واقعیت چیز دیگری‌ست. دلیل بروز نبرد مربوط به یک يا چند عامل ساده نیست. ده‌ها و یا شاید صدها علت کوچک و بزرگ باعث انباشت کینه‌ی ملت‌ها علیه یکدیگر می‌شود و آن زمان که وقت فوران خشم فرا می‌رسد. کسی نمی‌تواند مانع وقوع نبرد شود. همه چیز مانند بازی «دومینو» برهم می‌ريزد و حوادث زنجیروار رخ می‌دهد.  گم شدن یک ازن: ح ر کت یکت الاح جنگی بهوداخل خاک#کشور مقانل#شلنک یک گلوله: ترورایک شخصق ظهم! نحوه‌ی استفاده: از یک رودخانه‌ی مشترک, بندر مشت رک منبع آنرژی امشتر ک؛استناد به یک ادعای گهته ارضی, اعلام استقلال یک تژاد: تخت ستم. غلایق  ات نیز ووت روک اواته۱  نژادی» شوونیستی» راسیستی» ایدئولوژیکی و ده‌ها دلیل دیگر!  اما ملت‌های خسته از جنگ پس از 

In [20]:
with open('persian_cleaned_corpus.txt', 'w', encoding='utf-8') as f:
    for text in cleaned_texts:
        f.write(text + '\n')

We will save two versions of the corpus - one with stopwords and one without them 

In [21]:
# with open('persian_cleaned_corpus_with_stopwords.txt', 'w', encoding='utf-8') as f:
#     for text in cleaned_texts:
#         f.write(text + '\n')

## Stemming

Here is another method for stemming that can be applied instead of hazm stemming. 
[PersianStemmer can be fouund here]("https://github.com/htaghizadeh/PersianStemmer-Python")

In [22]:
# !pip install PersianStemmer
# !pip install https://github.com/htaghizadeh/PersianStemmer-Python/archive/master.zip --upgrade

In [23]:
# from PersianStemmer import PersianStemmer 

# # Define the stemmer
# stemmer = PersianStemmer()

# # Stem each text in the filtered_texts list
# stemmed_texts = []
# for text in cleaned_corp:
#     stemmed_text = ' '.join(stemmer.stem(w) for w in text.split())
#     stemmed_texts.append(stemmed_text)

# # Assign the stemmed texts to the original variable name
# cleaned_corp = stemmed_texts
